In [1]:
import numpy as np
from numba import cuda
import cupy as cp
from cupyx.time import repeat
from brownpy.utils import set_cuda_array
N=100_000
dtype=np.float32



In [10]:
def test1():
    a = np.zeros(N)
    d_a = cuda.to_device(a)
    d_a.copy_to_host(a)
print(repeat(test1, n_repeat=100, n_warmup=10))

C:\Users\monet\anaconda3\lib\site-packages\cupyx\time.py:115: FutureWarning: cupyx.time.repeat is experimental. The interface can change in the future.
  _util.experimental('cupyx.time.repeat')


test1               :    CPU: 1646.293 us   +/-454.376 (min: 1237.300 / max: 3237.400) us     GPU-0: 1681.179 us   +/-458.008 (min: 1252.352 / max: 3314.688) us


In [15]:
threadsperblock = 128
blockspergrid = int(np.ceil(N/threadsperblock))
a = np.empty(N)
d_a = cuda.to_device(a)
def test2():
    set_cuda_array[blockspergrid, threadsperblock](a, 0)
    d_a.copy_to_host(a)
print(repeat(test2, n_repeat=100, n_warmup=10))

test2               :    CPU: 2636.007 us   +/-832.954 (min: 1744.900 / max: 6426.300) us     GPU-0: 2676.962 us   +/-842.345 (min: 1767.552 / max: 6521.856) us


In [26]:
threadsperblock = 16
blockspergrid = int(np.ceil(N/threadsperblock))
p_a = cuda.pinned_array(shape = (N,), dtype=dtype)
def test3():
    set_cuda_array[blockspergrid, threadsperblock](p_a, 0)
    # d_a.copy_to_host(a)
print(repeat(test3, n_repeat=100, n_warmup=10))

test3               :    CPU: 1229.201 us   +/-442.802 (min:  889.200 / max: 3591.800) us     GPU-0: 1268.777 us   +/-456.851 (min:  887.808 / max: 3671.040) us


In [14]:
def test4():
    cp.zeros(shape=(N,)).get()
print(repeat(test3, n_repeat=100, n_warmup=10))
a

C:\Users\monet\anaconda3\lib\site-packages\cupyx\time.py:115: FutureWarning: cupyx.time.repeat is experimental. The interface can change in the future.
  _util.experimental('cupyx.time.repeat')


test3               :    CPU: 1133.985 us   +/-286.686 (min:  881.000 / max: 2619.600) us     GPU-0: 1169.197 us   +/-292.729 (min:  914.432 / max: 2660.352) us


array([9.12869372e-312, 9.12865377e-312, 0.00000000e+000, ...,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000])